In [22]:
# 广播: 是numpy对不同形状(shape)的数组进行数值计算的方式, 对数组的算数计算同查收那个在相应的元素上进行.

# 如果两个数组形状相同, 即满足a.shape==b.shape, 那么默认就是对应元素进行运算.

# 如果是形状不同的, 那么numpy会先使用广播机制,将两个数组进行广播, 变成形状相同, 然后进行运算.

import numpy as np


a = np.arange(12)
a.shape = (4, 3)
b = np.arange(3)
c = a+b
print(c) # 相加成功.
# b先广播到a的形状, 然后进行对应元素相加.



[[ 0  2  4]
 [ 3  5  7]
 [ 6  8 10]
 [ 9 11 13]]


In [23]:
# 相当于:
bb = np.tile(b, (4, 1)) # 将b为一个unit, 复制4行, 1列
# np.tile(a, reps) # 就是将a作为单元, 复制reps的形状.
print(bb)
print("-" * 5)
print(a  + bb)

[[0 1 2]
 [0 1 2]
 [0 1 2]
 [0 1 2]]
-----
[[ 0  2  4]
 [ 3  5  7]
 [ 6  8 10]
 [ 9 11 13]]


In [24]:
# 广播的规则:
# 所有输入数组向形状最长的数组看起, 形状不足的部分都通过在前面加1补齐.
# 输出数组的形状是输入数组形状的各个维度上的最大值
# 如果输入数组的某个维度和输出数组的对应维度的长度相同或者其长度为1时, 这个数组能够用来计算, 否则报错
# 当输入数组的某个维度长度为1时, 沿着此维度运算时都用此维度上的第一组值.
# 简单理解:
# 对比两个数组, 分别比较他们的每隔一个维度, 满足以下, 则可以使用广播机制
# 数组拥有相同形状
# 当前维度的值相等
# 当前维度的值有一个是1.

In [34]:
# 迭代
# numpy.nditer基本使用
# 他是一个有效的多维迭代器对象, 可以在数组上进行迭代. 数组的每个元素都可以使用python的标准iterator接口来访问.

import numpy as np

a = np.arange(12)
a = a.reshape(3, 4)

print(a)

for x in np.nditer(a):
    print(x, end=', ')

print("\n")

# 不是使用C或者Fortan顺序, 选择的顺序是和数组内存布局一致的, 这样做是为了提供访问的效率, 默认是行序有限(row-major order)或者说是C-order, 反映了默认情况下只需要访问每个元素, 而无需考虑其特定的顺序. 可以通过迭代上述的转置数组, 并且以C顺序访问数组转置的copy的方式进行对比.
# a和a.T的遍历顺序是一样的, 因为在内存中他们的存储顺序也是一样的,但是a.T.copy(order='C')的遍历结构是不同的, 因为它和前两种的存储方式是不一样的, 默认是按行访问.



[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 



In [36]:
for x in np.nditer(a.T):
    print(x, end=',')
print("\n")
print(a.T)

print("---"*10)
b = a.T.copy(order='C') # 重新深拷贝, 以'C'
print(b)
print("\n")

# 得到了一个新的数组, 在内存中的存放方式就与上面不同.
for x in np.nditer(b):
    print(x, end=',')

0,1,2,3,4,5,6,7,8,9,10,11,

[[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]
------------------------------
[[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]


0,4,8,1,5,9,2,6,10,3,7,11,

In [41]:
# 控制遍顺序: 也就是行优先, 还是列优先.
# Fortran order 列序有限: order='F'
# C order 行序优先: order='C'
# 注意, 一种是存储方式, 是按照行,还是列优先存储, 另一种就是遍历的方式, 是行还是列优先.
print(a)
print("\n")

b = a.T.copy(order="F") # 列优先存储
print(b) # 转置,
for x in np.nditer(b):
    print(x, end=', ') # 因为是列优先存储, 所以遍历是按照内存中的.
print("\n")

for x in np.nditer(b, order='C'): # 按照列来遍历
    print(x, end=', ')
print("\n")

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


[[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 

0, 4, 8, 1, 5, 9, 2, 6, 10, 3, 7, 11, 



In [46]:
# 那么我们在遍历的时候修改元素:
print(a)
print("\n")

for x in np.nditer(a, order='F', op_flags=["readwrite"]):
    x[...] = x * 2 # 这样才能写.
# print("\n")
print(a)

[[ 0  2  4  6]
 [ 8 10 12 14]
 [16 18 20 22]]


[[ 0  4  8 12]
 [16 20 24 28]
 [32 36 40 44]]


In [49]:
# 外部循环:
# nditer类的构造器拥有flags参数, 他接受以下值:
# c_index: 可以跟踪C顺序的索引
# f_index: 可以跟踪Fortran顺序的索引
# multi-index: 每次迭代可以跟踪一种索引类型
# external_loop: 给出的值具有多个值的一维数组, 而不是零维数组.
#   也就是遍历出来的值是一个数组, 而不是数据.

a = np.arange(12)
a = a.reshape(3, 4)
for x in np.nditer(a, flags=['external_loop'], order='F'):
    print(x, end=', ')
    # 按照列遍历, 将那个列当成元组对待.



[0 4 8], [1 5 9], [ 2  6 10], [ 3  7 11], 

In [54]:
# 广播迭代: 如果两个数组是可广播的, nditer组合对象能够同时迭代它们. 假设数组a具有维度3*4, 并且存在1*4的另一个数组b, 则使用以下类型的迭代器,(数组b被广播到a的大小).
a = np.arange(12).reshape(3, 4)

print(a)

b = np.arange(1, 5)
print(b)

print([a, b])


for x, y in np.nditer([a, b]): # 前提a,b是可以广播的.
    print("%d:%d" % (x, y), end=', ')

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[1 2 3 4]
[array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]]), array([1, 2, 3, 4])]
0:1, 1:2, 2:3, 3:4, 4:1, 5:2, 6:3, 7:4, 8:1, 9:2, 10:3, 11:4, 

In [2]:
# 继续数组的修改。
# 数组修改形状
# Ndarray.reshape(shape, order='C')
# order: C行, F列, A原顺序, k元素在内存中的出现顺序
import numpy as np

a = np.arange(12)
b = a.reshape((3, 4))
print(b)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


In [4]:
# flat: 一个数组元素的迭代器.
# Ndarray.flat
a = np.arange(12).reshape((3, 4))
print(a)
for x in a.flat:
    print(x) # 进行迭代.


[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
0
1
2
3
4
5
6
7
8
9
10
11


In [10]:
# Ndarray.flatten(order='C')
# 展平数组元素并且拷贝一份, 通常是按照C风格. 注意是拷贝, 而不是a的视图.
# order: 参数和作用和上面一样.
a = np.arange(12).reshape((3, 4))
print(a)
print("\n")

b = a.flatten() # copy了一份, flatten后给b.
b[0] = 100

print(b)

c = a.flatten(order='F')
print(c)




[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


[ 0  1  2  3  4  5  6  7  8  9 10 11]
[ 0  4  8  1  5  9  2  6 10  3  7 11]


In [13]:
# ravel()
# 展平的数组元素, 顺序通常是按照C, 返回的是数组的视图(View, 有点类似于C/C++引用reference的意味)
# View, 就有点类似reference. 有点像映射. 有点像起别名的操作.
# 注意, 修改会影响原来的数组. 因为返回的是view. 所以会影响原始的数组.

a = np.arange(12).reshape((3, 4))

b = a.ravel() # 创建了一个视图给b. 实际上,b和a指向的是同一个内存地址.
print(b)
b[0] = 100
print(b)
print(a)
# 可见, a和b的地方都改变了.


# 但是我们来看这样一种情况,
c = a.ravel(order='F') # 此时涉及到内存地址了, 所以是重新copy了一份, 然后再将view给c
print(c)
c[1] = 100
print(c)
print(a)
# 这也是比较特殊的地方.

[ 0  1  2  3  4  5  6  7  8  9 10 11]
[100   1   2   3   4   5   6   7   8   9  10  11]
[[100   1   2   3]
 [  4   5   6   7]
 [  8   9  10  11]]
[100   4   8   1   5   9   2   6  10   3   7  11]
[100 100   8   1   5   9   2   6  10   3   7  11]
[[100   1   2   3]
 [  4   5   6   7]
 [  8   9  10  11]]


In [16]:
# 翻转数组:
# 四种方式实现数组的翻转
# transpose(a, axes=None)
# 对换数组的维度,
# a: 需要操作的数组.
# axes整数列表, 对应维度, 通常所有维度都要对换
# 注意: 修改会影响原始数组.

import numpy as np

a = np.arange(12).reshape((3, 4))
print(a)
print("\n")

b = np.transpose(a)
print(b) # 转置了.


[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


[[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]


In [19]:
# ndarray.T
# 属性, 求转置, 类似于transpose.
a = np.arange(12).reshape((3, 4))
print(a)
print("\n")

b = a.T # 同样返回的是view
print(b)
b[0] =100 #

print(a)
print(b)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


[[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]
[[100   1   2   3]
 [100   5   6   7]
 [100   9  10  11]]
[[100 100 100]
 [  1   5   9]
 [  2   6  10]
 [  3   7  11]]


In [23]:
# rollaxis()
# numpy.rollaxis(a, axis, start=0)
# 作用: 向后滚动特定的轴体到一个特定的位置.
# axis: 需要向后滚动的轴, 其他轴的相对位置不会改变.
# start=0, 表示完整的滚动, 会滚动到特定位置.


a = np.arange(24).reshape((2, 3, 4))
print(a)
print(a.shape)
print("\n")

b = np.rollaxis(a, 1) # 将axis=1, 滚动第二轴, 滚动到start的位置.
print(b)
print(b.shape)

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
(2, 3, 4)


[[[ 0  1  2  3]
  [12 13 14 15]]

 [[ 4  5  6  7]
  [16 17 18 19]]

 [[ 8  9 10 11]
  [20 21 22 23]]]
(3, 2, 4)


In [24]:
# swapaxes()
# numpy.swapaxes(a, axis1, axis2)
# 交换数组两个轴
a = np.arange(24).reshape((2, 3, 4))
print(a)
print(a.shape)
print("\n")


b = np.swapaxes(a, 0, 2)
print(b.shape)

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
(2, 3, 4)


(4, 3, 2)
